## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
def read_data(path='../data/news_train.txt'):
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            category, title, text = line.strip().split('\t')
            yield category, title, text

In [6]:
import re
import os

newses = []
lines = list(open('../data/news_train.txt', 'r', encoding='utf-8'))

for category, title, text in read_data():
    news = []
    matches = re.findall(r'\b\w+\b', title.lower())

    for match in matches:
        news.append(match)
    newses.append(news)
    matches = re.findall(r'\b\w+\b', text.lower())
    news = []
    for match in matches:
        news.append(match)
    newses.append(news)


In [7]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
for news in newses:
    for i, new in enumerate(news):
        news[i] = morph.parse(new)[0].normal_form



In [8]:
from gensim.models import Word2Vec
import numpy as np

w2v = Word2Vec(newses, workers=8)
w2v.wv.save_word2vec_format('../data/w2v_vectors.bin')

In [9]:
w2v.wv.most_similar(positive=["июнь"])


[('август', 0.9766823649406433),
 ('июль', 0.9730883240699768),
 ('апрель', 0.9697406888008118),
 ('май', 0.9660383462905884),
 ('февраль', 0.9647614359855652),
 ('март', 0.9644171595573425),
 ('сентябрь', 0.9640851020812988),
 ('ноябрь', 0.9578044414520264),
 ('декабрь', 0.9548572897911072),
 ('октябрь', 0.95158851146698)]

In [2]:
import numpy as np

def prepare_data(path):
    lines = list(open(path, 'r', encoding='utf-8'))

    categories = { category for (category, _, _) in read_data(path)}
    categories = { category:i for (i, category) in enumerate(categories) }

    X = []
    y = []

    for category, title, text in read_data(path):
        X.append(title + ' ' + text)
        y.append(categories[category])
    return np.array(X), np.array(y)

In [3]:
path_train = '../data/news_train.txt'
path_test = '../data/news_test.txt'

X_train, y_train = prepare_data(path_train)
X_test, y_test = prepare_data(path_test)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(n_jobs=-1)
tfidf = TfidfVectorizer()
text_clf = Pipeline([
                     ('tfidf', tfidf),
                     ('clf', clf)
                     ])


text_clf.fit(X_train, y_train)
X_test_idf = tfidf.transform(X_test)
preds = clf.predict(X_test_idf)

print('Точность', (y_test == preds).mean())

Точность 0.8743333333333333


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
tfidf = TfidfVectorizer()

text_clf = Pipeline([
                     ('tfidf', tfidf),
                     ('clf', clf)
                     ])


text_clf.fit(X_train, y_train)
X_test_idf = tfidf.transform(X_test)
preds = clf.predict(X_test_idf)

print('Точность', (y_test == preds).mean())

Точность 0.808
